In [41]:
from transformers import AutoModel, AutoTokenizer, BertLMHeadModel, BertConfig
model_name = "huggingface/CodeBERTa-small-v1" 

encoder1 = AutoModel.from_pretrained(model_name)
encoder2 = AutoModel.from_pretrained(model_name)
config = BertConfig.from_pretrained("bert-base-uncased")
config.is_decoder = True
config.add_cross_attention=True
config.output_hidden_states=True
decoder = BertLMHeadModel.from_pretrained("bert-base-uncased", config=config)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at huggingface/CodeBERTa-small-v1 were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at huggingface/CodeBERTa-small-v1 were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.bi

In [42]:
import torch
zeros = torch.zeros(1, 1, 768)
zeros.shape

torch.Size([1, 1, 768])

In [43]:
input = tokenizer.encode_plus("Hello world!", return_tensors = "pt")
output = encoder1(**input)
encoder_hidden_state = output[0]
decoder_output = decoder(inputs_embeds=zeros, encoder_hidden_states=encoder_hidden_state)
decoder_output.keys()

odict_keys(['logits', 'past_key_values', 'hidden_states'])

In [73]:
p_gen = torch.sigmoid(decoder_output['logits'])
softmax = torch.nn.Softmax(dim=2)
p_vocab = softmax(decoder_output['logits'])
p_gen.shape, p_vocab.shape

(torch.Size([1, 1, 30522]), torch.Size([1, 1, 30522]))

In [76]:
(p_gen*p_vocab).shape

torch.Size([1, 1, 30522])

In [74]:
decoder_output['hidden_states'][12].shape

torch.Size([1, 1, 768])

In [78]:
#attention_probs 
context_layer = decoder.bert.encoder.layer[11].attention.self(zeros)[1]

In [80]:
context_layer[1].shape

torch.Size([1, 12, 1, 64])

In [26]:
decoder.bert.encoder.layer[11].attention.self(zeros)

(tensor([[[ 1.5023e-02, -5.3094e-03,  2.3572e-04,  2.0522e-03, -5.7804e-03,
            8.1983e-03, -1.1096e-02, -4.4216e-03,  1.0920e-03, -5.7501e-03,
           -1.2049e-03,  3.3312e-03, -5.7881e-03,  3.2863e-03,  1.6009e-02,
           -3.8654e-03,  7.0702e-03,  9.0583e-03,  9.7348e-03, -8.1410e-03,
            1.1158e-02, -5.4021e-03, -1.4808e-02, -6.7811e-03,  5.0396e-03,
            4.8309e-03,  1.2264e-02,  1.7033e-03,  3.8622e-03, -5.0164e-03,
            1.0777e-02,  2.2679e-03,  1.2602e-02,  1.7358e-03, -8.8981e-03,
           -1.6617e-03, -5.4248e-03,  1.5820e-02, -2.7107e-03,  9.3095e-03,
            1.4642e-02,  1.8524e-03, -6.2581e-03,  1.3867e-02,  1.6465e-02,
           -9.9101e-03, -5.9330e-03,  3.1781e-03,  6.2056e-03, -5.7156e-03,
            1.5173e-02,  8.2161e-03, -7.5105e-03, -6.8956e-03,  1.1878e-02,
            1.2468e-02, -1.9451e-03,  1.4344e-02,  3.4198e-03,  9.5356e-03,
            1.9738e-03, -2.3401e-03,  2.2367e-03, -9.8477e-04,  2.0704e-02,
            

In [20]:
decoder.bert.encoder.layer[11].attention.self(zeros)[1].shape

torch.Size([1, 1, 768])

In [70]:
inputs

{'input_ids': tensor([[    0, 15194, 12015,     5,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [81]:
decoder

BertLMHeadModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [43]:
decoder(outputsa)

AttributeError: 'BaseModelOutputWithPoolingAndCrossAttentions' object has no attribute 'size'

In [41]:
inputs = tokenizer("Hello world!", return_tensors="pt")
outputs = encoder1(**inputs)
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.2280,  0.7934, -0.3273,  ..., -0.1060,  0.5652, -0.4171],
         [ 0.7355, -0.4856,  0.2469,  ...,  0.1477,  0.7043,  0.2593],
         [ 1.3592,  0.8465, -0.0296,  ...,  0.0119,  0.0665,  0.2345],
         [ 0.8412,  0.5825, -0.8065,  ..., -0.0471,  0.9530, -0.4562],
         [ 0.2643,  0.9847, -0.6103,  ...,  1.0001,  0.0587, -0.3000]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[-1.5272e-01,  1.4222e-01, -2.3856e-01,  5.7524e-01, -4.6813e-01,
         -2.1461e-01,  3.3826e-01, -1.6099e-01, -2.0227e-01,  5.3433e-01,
          1.9483e-01, -2.2482e-01, -9.0089e-01,  8.2799e-02,  4.2339e-01,
         -5.7091e-01,  1.7547e-01, -6.0339e-01, -5.3633e-01,  8.4052e-02,
          6.8741e-01, -5.4140e-01,  1.9744e-01,  7.7146e-01, -9.9504e-04,
          4.2471e-01, -5.5083e-01, -5.6025e-01,  5.4110e-01,  3.0390e-01,
          3.9060e-02, -3.4018e-01,  6.5153e-02, -2.8344e-01, -2.9998e-02,
     

In [10]:
inputs['input_ids']

tensor([[    0, 15194, 12015,     5,     2]])

In [16]:
outputs['last_hidden_state'].shape

torch.Size([1, 5, 768])

In [29]:
from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel
# Initializing a BERT bert-base-uncased style configuration
config_encoder = BertConfig(num_hidden_layers=2)
config_decoder = BertConfig(num_hidden_layers=2)
config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)
# Initializing a Bert2Bert model from the bert-base-uncased style configurations
model = EncoderDecoderModel(config=config)
# Accessing the model configuration
config_encoder = model.config.encoder
config_decoder  = model.config.decoder
# set decoder config to causal lm
config_decoder.is_decoder = True
config_decoder.add_cross_attention = True

In [30]:
model.decoder

BertLMHeadModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
#concat them=2 NN: Attention based later

